Step 6 - Extract the most important content from caption text

The most important words from dialogues can in some ways show the gist of what a game is talking about. This script is used to extract some of such words using nltk.

Prerequisites: nltk

Input:

1. caption text

Output:

1. A list of words and their corresponding vectors

In [1]:
%pylab
import os, glob
import json
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Zooerius\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Zooerius\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Zooerius\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [67]:
"""
Make sure these variables are correctly set.
caption_file: a directory which contains caption files
answer_file: file path to answer file
"""

caption_path = './data source/Life Is Strange 1/output'
output_file = './data source/Life Is Strange 1/output/tf-idf.json'

In [68]:
caption_files = sorted(glob.glob(os.path.join(caption_path, '**/*.txt'), recursive=True))  
print(f'Reading caption files...')
corpus = {}
for index, file in enumerate(caption_files):
    print(f'File {index}: {file}')
    with open(file, 'r') as txt:
        corpus[index] = txt.read()
        print(corpus[index], '\n')


Reading caption files...
File 0: ./data source/Life Is Strange 1/output\Episode_1_Chrysalis\screenshots\caption.txt
you happening and where is here wait there's the lighthouse I hope please let me make it there holy shit wow that was so cerejo mostly called film little pieces of time but he could be talking about photography as he likely was okay in class everything's cool I am okay from light to shadow from color to chiaroscuro now can you give me an example of a photographer who perfectly captured the human condition in he didn't fall asleep and it sure didn't feel like a dream Bueller weird Diane Arbus there you go Victoria why Arvest because of her look at this crap how can I show this to mr. Jefferson I can hear the class laughing at me now images of hopeless faces you feel like totally haunted by the eyes of those sad mothers and children she saw humanity is torture a little camera bag is battered but still kick you should keep that to yourself if anybody else looked at this what

In [69]:
def tokenize(text):
    stop_words = stopwords.words('english')
    stop_words += ['yeah', 'okay', 'yes', 'hey', 'huh', "let", "something", "everything", "nothing", "anything", "thing", "guy", "alright"]
    stop_words = set(stop_words)
    lemmatizer = nltk.WordNetLemmatizer()
    
    mapping = [ ('gotta', 'have got to'), 
               ('gonna', 'going to')]
    for k, v in mapping:
        text = text.replace(k, v)
    
    word_tokens = text.split(' ') # word_tokenize(text)
    
    words = [lemmatizer.lemmatize(word) for word in word_tokens]
    words = [word for word in words if word.isalpha()]
    words = [word for word in words if word.lower() not in stop_words]
    words = [word for word in words if len(word)> 2]
    return words

In [70]:
# calculate the frequency of each word
text = ' '.join(corpus.values())
tokens = tokenize(text)
total_num = len(tokens)
fd = FreqDist(tokens)
topn_words = dict(fd.most_common())
print(f'{len(tokens)} tokens in total.', f'{len(topn_words.keys())} unique words.', topn_words, sep='\n')

24447 tokens in total.
3743 unique words.
{'max': 478, 'know': 382, 'like': 378, 'Chloe': 281, 'get': 279, 'see': 207, 'Kate': 181, 'want': 174, 'Nathan': 174, 'right': 172, 'going': 169, 'time': 163, 'look': 159, 'think': 158, 'Rachel': 157, 'one': 140, 'even': 134, 'need': 132, 'would': 130, 'could': 118, 'way': 118, 'come': 116, 'shit': 113, 'make': 111, 'sorry': 111, 'Victoria': 109, 'back': 109, 'good': 107, 'find': 97, 'take': 97, 'tell': 92, 'still': 91, 'feel': 89, 'better': 88, 'please': 86, 'talk': 86, 'Frank': 86, 'Jefferson': 85, 'well': 85, 'David': 84, 'friend': 84, 'much': 83, 'always': 82, 'never': 78, 'cool': 73, 'photo': 73, 'got': 73, 'Blackwell': 73, 'believe': 72, 'life': 70, 'thanks': 70, 'maybe': 70, 'party': 70, 'really': 67, 'fucking': 66, 'say': 66, 'love': 66, 'real': 65, 'help': 64, 'Warren': 64, 'sure': 63, 'glad': 63, 'hope': 62, 'around': 62, 'mean': 62, 'Prescott': 61, 'ever': 57, 'Arcadia': 57, 'care': 56, 'Bay': 56, 'stop': 55, 'people': 55, 'little': 

In [71]:
# find nouns

tags = nltk.pos_tag(topn_words.keys())
nouns = [tag[0] for tag in tags if tag[1] in ['NNP', 'NN', 'NNS', 'NNPS']]

In [72]:
# calculate idf
chunk_size = 100
chunks = [tokens[x:x+chunk_size] for x in range(0, len(tokens), chunk_size)]

idf={}
tfidf={}
for term in nouns:
    freq = float(topn_words[term])
    idf[term] = math.log(float(len(chunks))/sum([1 for doc in chunks if term in doc]), 10)
    tfidf[term] = freq * idf[term]
    
tfidf_sorted = sorted(tfidf.items(), key=lambda x: x[1], reverse=True)

print(tfidf_sorted[:20])
print(tfidf_sorted[len(tfidf_sorted)-20:])

[('Kate', 81.38607654601041), ('Chloe', 79.2296682353826), ('Frank', 76.02538511983788), ('Rachel', 74.63030042098806), ('Nathan', 74.01581672130203), ('Victoria', 69.86660825205819), ('David', 65.21610712216294), ('Jefferson', 59.41245036856159), ('sorry', 55.174934465819774), ('way', 53.65078070827384), ('time', 52.927419507431075), ('party', 52.89883401494621), ('make', 51.602434611594234), ('find', 51.00679275167542), ('talk', 49.557734584064214), ('tell', 49.48751167938205), ('photo', 48.53698567775327), ('Warren', 48.364648242236534), ('life', 47.71171357866172), ('please', 47.32726145194582)]
[('Terry', 2.3891660843645326), ('grunge', 2.3891660843645326), ('quality', 2.3891660843645326), ('shortcut', 2.3891660843645326), ('suffering', 2.3891660843645326), ('Stockholm', 2.3891660843645326), ('Syndrome', 2.3891660843645326), ('Rahsaan', 2.3891660843645326), ('voteban', 2.3891660843645326), ('dicey', 2.3891660843645326), ('icon', 2.3891660843645326), ('parliament', 2.38916608436453

In [73]:
with open(output_file, 'w+') as output:
    json.dump(tfidf, output, sort_keys=False, indent=4)

In [ ]:
"""

CC coordinating conjunction
CD cardinal digit
DT determiner
EX existential there (like: “there is” … think of it like “there exists”)
FW foreign word
IN preposition/subordinating conjunction
JJ adjective ‘big’
JJR adjective, comparative ‘bigger’
JJS adjective, superlative ‘biggest’
LS list marker 1)
MD modal could, will
NN noun, singular ‘desk’
NNS noun plural ‘desks’
NNP proper noun, singular ‘Harrison’
NNPS proper noun, plural ‘Americans’
PDT predeterminer ‘all the kids’
POS possessive ending parent’s
PRP personal pronoun I, he, she
PRP$ possessive pronoun my, his, hers
RB adverb very, silently,
RBR adverb, comparative better
RBS adverb, superlative best
RP particle give up
TO, to go ‘to’ the store.
UH interjection, errrrrrrrm
VB verb, base form take
VBD verb, past tense took
VBG verb, gerund/present participle taking
VBN verb, past participle taken
VBP verb, sing. present, non-3d take
VBZ verb, 3rd person sing. present takes
WDT wh-determiner which
WP wh-pronoun who, what
WP$ possessive wh-pronoun whose
WRB wh-abverb where, when

"""
words = ['max', 'know', 'like', 'get', 'Chloe', 'Max', 'David', "nathan"]

nltk.pos_tag(words)
